https://wybory.gov.pl/pe2019/pl/wyniki/protokol/485230

https://selenium-python.readthedocs.io/api.html

https://stackoverflow.com/questions/23110383/how-to-dynamically-build-a-json-object-with-python

In [32]:
import pandas as pd
from selenium import webdriver
import os

projectDirectory = "/home/lukek/Projects/github/lukasz-kusek/european-elections-2019-poland"

In [22]:
options = webdriver.ChromeOptions() 
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [33]:
obwody_glosowania = pd.read_csv(os.path.join(projectDirectory, 'dane_statyczne', 'obwody_glosowania.csv'), sep = ';')

In [34]:
TERYT_codes = obwody_glosowania['TERYT gminy'].unique()
print(TERYT_codes)

[ 20101  20102  20103 ... 326101 326201 326301]


In [35]:
def get_protocols_ids(gmina_id):
    driver.get('https://wybory.gov.pl/pe2019/pl/wyniki/gm/' + gmina_id)
    results = driver.find_elements_by_css_selector("div.proto tbody tr")
    return list(map(lambda elem: elem.get_attribute("data-id"), results))
    
print(get_protocols_ids('20101'))

['457895', '457896', '457897', '457898', '457899', '457900', '457901', '457902', '457903', '457904', '457905', '457906', '457907', '457908', '457909', '457910', '457911', '457912', '457913', '457914', '457915', '485228', '485229', '485230']


In [55]:
def extract_card(card_web_element):
    tds = card_web_element.find_elements_by_css_selector('td')
    card = {}
    card['id'] = tds[0].text
    card['name'] = tds[1].text
    card['count'] = tds[2].text
    return card

def get_protocol_data(protocol_id):
    driver.get('https://wybory.gov.pl/pe2019/pl/wyniki/protokol/' + protocol_id)
    info = driver.find_element_by_xpath('//div[contains(@class, "pro")]/dl[contains(@class, "info")]')
    print (info.get_attribute('outerHTML'))
    cards = driver.find_elements_by_css_selector("div.cards table tbody tr")
    cards = list(map(lambda card: extract_card(card), cards))
    lists = driver.find_element_by_css_selector("div.lists")
    print (lists.get_attribute('outerHTML'))
    
    protocol = {}
    protocol['info'] = info
    protocol['cards'] = cards
    protocol['lists'] = lists
    return protocol
print(get_protocol_data('457895'))

<dl class="dl-horizontal info"><dt>Siedziba</dt><dd>Szkoła Podstawowa Nr 3, ul. Ceramiczna 5, 59-700 Bolesławiec</dd><dt>Numer</dt><dd>1</dd><dt>Okręg wyborczy</dt><dd><span style="display:none">012</span>Nr 12 [Wrocław]</dd><dt>Kod TERYT</dt><dd>020101</dd><dt>Województwo</dt><dd>dolnośląskie</dd><dt>Powiat</dt><dd>bolesławiecki</dd><dt>Gmina</dt><dd>m. Bolesławiec</dd></dl>
<div class="lists"><div><h3>Nr listy 1 - KOMITET WYBORCZY WYBORCÓW KONFEDERACJA KORWIN BRAUN LIROY NARODOWCY</h3><div class="table-responsive"><table class="table table-bordered table-striped table-hover right3"><thead><tr><th>Numer</th><th>Nazwisko i imiona</th><th>Liczba głosów</th></tr></thead><tbody><tr><td>1</td><td>IWASZKIEWICZ Robert Jarosław</td><td><div class="hidden">0000000012</div>12</td></tr><tr><td>2</td><td>ŻUREK Piotr Grzegorz</td><td><div class="hidden">0000000000</div>0</td></tr><tr><td>3</td><td>POPLICHA Dagmara Ewa</td><td><div class="hidden">0000000000</div>0</td></tr><tr><td>4</td><td>SZKARUP